In [ ]:
import pandas as pd
import numpy as np
import glob
import ast
import re
import os
import time
import json


from openai import OpenAI
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GroupShuffleSplit
from statsmodels.stats.inter_rater import fleiss_kappa
from statsmodels.stats import inter_rater as irr
from collections import Counter


In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
client = OpenAI()

# Test and train sets creation

In [ ]:



# Path to the root directory (change to your root directory)
root_dir = "~/annotations"

# Subdirectories where the Excel files are stored
subdirs = ["A", "B", "C", "D"]

# Collect all Excel files from the subdirectories
excel_files = []
for subdir in subdirs:
    path = os.path.join(root_dir, subdir, "12", "*.xlsx")
    excel_files.extend(glob.glob(path))




# Initialize an empty list to store the new rows for the DataFrame
all_data = []

# Function to extract document ID
def extract_id(filename):
    match = re.search(r"Annotation_2018argrewrite_(\d+)", filename)
    return match.group(1) if match else "unknown"


# Function to handle Aligned Index and retrieve corresponding sentence(s)
def get_text_after(aligned_index_str : str) -> str:
    if aligned_index_str == "DELETE":
        return ""  # Empty string for DELETE
    try:
        # Convert string like '1, 2, 3' to a list of integers
        aligned_indices = list(map(int, aligned_index_str.split(",")))
    except ValueError:
        return None  # If conversion fails, return None

    # Find corresponding "Sentence Content" in new_draft
    text_after = []
    for index in aligned_indices:
        matching_rows = new_draft_df[new_draft_df["Sentence Index"] == index]
        for _, row in matching_rows.iterrows():
            text_after.append(row["Sentence Content"])

    return " ".join(text_after)  # Join sentences if there are multiple matches


for file in excel_files:
        # Read both sheets
    old_draft_df = pd.read_excel(file, sheet_name="Old Draft")
    new_draft_df = pd.read_excel(file, sheet_name="New Draft")

    doc_id = extract_id(os.path.basename(file))  # Extract document ID
    # Iterate over the rows in the old draft
    for _, old_row in old_draft_df.iterrows():
        # Check if "Identical?" == 0
        if old_row["Identical?"] == 0:
            text_before = old_row["Sentence Content"]  # Sentence from old draft
            aligned_index = old_row["Aligned Index"]  # Aligned Index in old draft
            label = old_row["Revision Purpose Level 0"]
            
            # Get corresponding "Text After" based on Aligned Index
            text_after = get_text_after(str(aligned_index))  # Ensure it's a string
            
            # Add the new row to the new_data list
            all_data.append({
                "Text Before": text_before,
                "Text After": text_after,
                "Label": label,
                "Doc Id": doc_id
            })
    for _, new_row in new_draft_df.iterrows():
        if str(new_row["Aligned Index"]) == "ADD":
            text_before = ""
            label = new_row["Revision Purpose Level 0"]
            text_after = new_row["Sentence Content"]

            all_data.append({
                "Text Before": text_before,
                "Text After": text_after,
                "Label": label,
                "Doc Id": doc_id
            })
# Create the new DataFrame
merged_df_12 = pd.DataFrame(all_data)

# Display or save the new DataFrame
# merged_df.to_excel("merged_output.xlsx", index=False)
print(merged_df_12.head())


In [ ]:


# Path to the root directory (change to your root directory)
root_dir = "/home/nbl/llm-notebooks/data-bea/annotations"

# Subdirectories where the Excel files are stored
subdirs = ["A", "B", "C", "D"]

# Collect all Excel files from the subdirectories
excel_files = []
for subdir in subdirs:
    path = os.path.join(root_dir, subdir, "23", "*.xlsx")
    excel_files.extend(glob.glob(path))




# Initialize an empty list to store the new rows for the DataFrame
all_data = []

# Function to extract document ID
def extract_id(filename):
    match = re.search(r"Annotation_2018argrewrite_(\d+)", filename)
    return match.group(1) if match else "unknown"


# Function to handle Aligned Index and retrieve corresponding sentence(s)
def get_text_after(aligned_index_str : str) -> str:
    if aligned_index_str == "DELETE":
        return ""  # Empty string for DELETE
    try:
        # Convert string like '1, 2, 3' to a list of integers
        aligned_indices = list(map(int, aligned_index_str.split(",")))
    except ValueError:
        return None  # If conversion fails, return None

    # Find corresponding "Sentence Content" in new_draft
    text_after = []
    for index in aligned_indices:
        matching_rows = new_draft_df[new_draft_df["Sentence Index"] == index]
        for _, row in matching_rows.iterrows():
            text_after.append(row["Sentence Content"])

    return " ".join(text_after)  # Join sentences if there are multiple matches


for file in excel_files:
        # Read both sheets
    old_draft_df = pd.read_excel(file, sheet_name="Old Draft")
    new_draft_df = pd.read_excel(file, sheet_name="New Draft")

    doc_id = extract_id(os.path.basename(file))  # Extract document ID
    # Iterate over the rows in the old draft
    for _, old_row in old_draft_df.iterrows():
        # Check if "Identical?" == 0
        if old_row["Identical?"] == 0:
            text_before = old_row["Sentence Content"]  # Sentence from old draft
            aligned_index = old_row["Aligned Index"]  # Aligned Index in old draft
            label = old_row["Revision Purpose Level 0"]
            
            # Get corresponding "Text After" based on Aligned Index
            text_after = get_text_after(str(aligned_index))  # Ensure it's a string
            
            # Add the new row to the new_data list
            all_data.append({
                "Text Before": text_before,
                "Text After": text_after,
                "Label": label,
                "Doc Id": doc_id
            })
            
    for _, new_row in new_draft_df.iterrows():
        if str(new_row["Aligned Index"]) == "ADD":
            text_before = ""
            label = new_row["Revision Purpose Level 0"]
            text_after = new_row["Sentence Content"]

            all_data.append({
                "Text Before": text_before,
                "Text After": text_after,
                "Label": label,
                "Doc Id": doc_id
            })
# Create the new DataFrame
merged_df_23 = pd.DataFrame(all_data)
label_counts = merged_df_23["Label"].value_counts()

# Filter out rows where the label count is 1
merged_df_23 = merged_df_23[merged_df_23['Label'].isin(label_counts[label_counts > 1].index)]

# Display or save the new DataFrame
# merged_df.to_excel("merged_output.xlsx", index=False)
print(merged_df_23.head())


In [ ]:
# Merge by concatenation
df = pd.concat([merged_df_12, merged_df_23], ignore_index=True)
df.to_csv("ArgRewrite_complete.csv", index=False)

In [56]:
df_filtered = df.groupby("Text After").filter(lambda group: group["Label"].nunique() == 1)
df_merged = df_filtered.groupby("Text After", as_index=False).agg({
    "Text Before": " ".join,  # Join text_before
    "Doc Id": "first",        # Keep the first author in each group
    "Label": "first"      # Keep the first timestamp in each group
})
df = df_merged

In [ ]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idxs, test_idxs = next(gss.split(df, groups=df["Doc Id"]))

train_df = df.iloc[train_idxs]
test_df = df.iloc[test_idxs]

print("Training set size:", len(train_df))
print("Test set size:", len(test_df))


In [ ]:
label_percentages = df["Label"].value_counts(normalize=True) * 100  # Convert to percentage
print(label_percentages)

In [ ]:
label_percentages = train_df["Label"].value_counts(normalize=True) * 100  # Convert to percentage
print(label_percentages)

In [ ]:
label_percentages = test_df["Label"].value_counts(normalize=True) * 100  # Convert to percentage
print(label_percentages)

# Essai 1 : Few-shot Cot 

In [ ]:
assistant_first = client.beta.assistants.create(
  name="ArgRewrite-try-1",
  instructions="""You are evaluating text revisions by comparing different states of a text (before and after a revision).

**Prompt students answered:**  
Write an op-ed for the *Pittsburgh City Paper* on whether self-driving cars benefit society. Briefly outline their advantages and disadvantages, then take a stance—either for or against—supporting it with strong evidence, explanations, and a counter-argument. Ensure clarity, precision, and proper organization.

**Classification Task:**  
Provide a step-by-step reasoning (Chain of Thought) before concluding with one of the following categories:

- **Word-Usage/Clarity**: Changes to improve clarity or word choice without altering the meaning of the content.
   - Example:  
      - Before: "Technology remains to be the nourishment of civilizations."  
      - After: "Technology remains to be the backbone of civilizations."

- **Conventions/Grammar/Spelling*: Corrections of spelling and grammatical errors.
- Example:  
      - Before: "Big corporations stick together and attack on one another."  
      - After: "Big corporations stick together and attack one another."

- **General Content Development**: Significant additions, deletions, or rewording of content without a clear argumentative function.
- Example:  
      - Before: "Self-driving cars will one day be the same."  
      - After: "Self-driving cars will one day be the same - once thought impossible, but eventually taken for granted."

- **Precision**: Edits that adjust the specificity of a sentence, influencing its level of detail.
- Example:  
      - Before: "However, the pros outweigh the cons."  
      - After: "However, the pros of self-driving cars outweigh the cons."

- **Organization**: Structural modifications, such as rearranging sentences by splitting them or reunite them.
- Example:  
      - Before: "In my opinion, I support self-driving cars. The technology will have positive impacts on the society."  
      - After: "In my opinion, I support self-driving cars because the technology will have positive impacts on the society."

- **Warrant/Reasoning/Backing**: Changes in explanations or logical support for a claim.
- Example:  
      - Before: "By reducing traffic, gas expenses would be lower."  
      - After: "Self-driving cars will have the ability to coordinate with each other to prevent and reduce traffic, which means gas expenses would be lower."

- **Evidence**: Additions or modifications of factual information to support an argument.
- Example:  
      - Before: "A lot of expensive machinery will be needed to make a self-driving car."  
      - After: "A lot of expensive machinery, like sensors and cameras, will be needed to make a self-driving car."


- **Claims/Ideas**: Edits that modify (add or delete) a main claim or idea of the essay.
- Example:  
      - Before: "Overall, self-driving cars are infiltrating our society at a rather rapid rate."  
      - After: "Overall, self-driving cars are immigrating into our society with many benefits."

- **Rebuttal/Reservation**: Edits related to refuting an opposing argument.
- Example:  
      - Before: "Self driving cars are the subject of much scrutiny."  
      - After: "Self driving cars are not perfect yet, and there are still many issues to be worked out."


**Guidelines for Ambiguous Cases:**  
If multiple revision purposes exist, follow these rules:  

- **Claim/Ideas vs. Warrant/Reasoning/Backing**: A main claim is "Claims/Ideas," while supporting arguments are "Warrant/Reasoning/Backing."  
- **General Content vs. Warrant/Reasoning/Backing**: If it expresses the author's stance, classify as "Warrant/Reasoning/Backing"; otherwise, as "General Content."  
- **Evidence vs. Warrant/Reasoning/Backing**: Factual support (e.g., citations, examples) is "Evidence"; otherwise, it's "Warrant/Reasoning/Backing."  
- **Conventions/Grammar/Spelling vs. Word-Usage/Clarity**: If only grammar/spelling is corrected, use "Conventions/Grammar/Spelling"; otherwise, use "Word-Usage/Clarity."  
- **Precision vs. Word-Usage/Clarity**: If a change affects meaning and specificity, use "Precision"; otherwise, "Word-Usage/Clarity."  
- **Claim/Idea vs. Word-Usage/Clarity**: If a change impacts the essay’s main argument, classify as "Claim/Ideas."  
- **Organization vs. General Content Development**: Reordering sentences is "Organization"; significant content changes are "General Content."  

You may consider previous text versions from the same essay for context.
 
**Output Format:**
Your response **must** follow this exact JSON format:  
```json
{
      "reasoning": "<Step-by-step explanation of the classification>",
      "classification": "category chosen"
}""",
  tools=[],
  model="gpt-4o",
)

# Essai 2 - Multiple steps

In [ ]:
assistant_first = client.beta.assistants.create(
  name="ArgRewrite-try-2-step1",
  instructions="""You are evaluating text revisions by comparing different states of a text (before and after a revision).

**Contextual Awareness:**  
You are evaluating a *sequence* of revisions within the same essay. It is essential to consider earlier parts of the essay to accurately determine the purpose of a revision. Always reason based on the surrounding context and how the revision fits into the overall argumentative structure.


**Prompt students answered:**  
Write an op-ed for the *Pittsburgh City Paper* on whether self-driving cars benefit society. Briefly outline their advantages and disadvantages, then take a stance—either for or against—supporting it with strong evidence, explanations, and a counter-argument. Ensure clarity, precision, and proper organization.


**Classification Task:**  
Provide a step-by-step reasoning (Chain of Thought) before concluding with one of the following categories for each text given as input:

- **Surface changes**: Edits that concerns surface changes like organization, spelling or grammar correction or clarity improvements that do not change the deep meaning of the text.
- Example:  
      - Before: "Technology is the nourishment of civilizations."  
      - After: "Technology is the backbone of civilizations." 

- Example:  
      - Before: "Big corporations stick together and attack on one another."  
      - After: "Big corporations stick together and attack one another."

- **Content changes**: Edits that concerns meaning changes directly linked to the argumentation or just general content development.
- Example:  
      - Before: "Self-driving cars are helpful."
      - After: "Self-driving cars reduce accidents by 90%."

- Example:  
      - Before: "Overall, self-driving cars are infiltrating our society at a rather rapid rate."  
      - After: "Overall, self-driving cars are immigrating into our society with many benefits."



You **must** consider previous messages in the thread as part of your context.

**Output Format:**
Your response **must** follow this exact JSON format:  
```json
{
      "reasoning": "<Step-by-step explanation of the classification>",
      "classification": "category chosen"
}""",
  tools=[],
  model="gpt-4o",
)

In [ ]:
assistant_first = client.beta.assistants.create(
  name="ArgRewrite-try-2-step2-content",
  instructions="""You are evaluating text revisions by comparing different states of a text (before and after a revision). The considered revision has been classified as having an impact on the meaning of the text. You should now subclassify it.

**Contextual Awareness:**  
You are evaluating a *sequence* of revisions within the same essay. It is essential to consider earlier parts of the essay to accurately determine the purpose of a revision. Always reason based on the surrounding context and how the revision fits into the overall argumentative structure.


**Prompt students answered:**  
Write an op-ed for the *Pittsburgh City Paper* on whether self-driving cars benefit society. Briefly outline their advantages and disadvantages, then take a stance—either for or against—supporting it with strong evidence, explanations, and a counter-argument. Ensure clarity, precision, and proper organization.


**Classification Task:**  
Provide a step-by-step reasoning (Chain of Thought) before concluding with one of the following categories:


- **General Content Development**: Significant additions, deletions, or rewording of content without a clear argumentative function.
- Example:  
      - Before: "Self-driving cars will one day be the same."  
      - After: "Self-driving cars will one day be the same - once thought impossible, but eventually taken for granted."


- **Precision**: Edits that add or reduce quantifiable, factual, or definitional detail, affecting how specific a statement is and altering its meaning. Use "Precision" only when an edit introduces or removes specific factual detail (e.g., names, quantities, types). If a change slightly broadens or narrows specificity without introducing or removing factual detail, classify it as "Word-Usage/Clarity."
- Example:  
      - Before: "Self-driving cars are helpful."
      - After: "Self-driving cars reduce accidents by 90%."

- **Warrant/Reasoning/Backing**: Changes in explanations or logical support for a claim, related to the prompt students answered to.
- Example:  
      - Before: "By reducing traffic, gas expenses would be lower."  
      - After: "Self-driving cars will have the ability to coordinate with each other to prevent and reduce traffic, which means gas expenses would be lower."

- **Evidence**: Additions or modifications of factual information to support an argument that is link to the prompt students answered to.
- Example:  
      - Before: "A lot of expensive machinery will be needed to make a self-driving car."  
      - After: "A lot of expensive machinery, like sensors and cameras, will be needed to make a self-driving car."


- **Claims/Ideas**: Edits that modify (add or delete) a main claim or idea of the essay related to the prompt students answered to. Make sure the revision is no related to a previous claim before choosing this type.
- Example:  
      - Before: "Overall, self-driving cars are infiltrating our society at a rather rapid rate."  
      - After: "Overall, self-driving cars are immigrating into our society with many benefits."

- **Rebuttal/Reservation**: Edits related to refuting an opposing argument.
- Example:  
      - Before: "Self driving cars are the subject of much scrutiny."  
      - After: "Self driving cars are not perfect yet, and there are still many issues to be worked out."


**Guidelines for Ambiguous Cases:**  
If you hesitate between multiple revision purposes, follow these rules:  

- **Precision vs. Word-Usage/Clarity**: Use "Precision" only if the revision introduces or removes **factual**, **technical**, or **quantifiable** detail. If the change improves clarity or adds rhetorical nuance (e.g., “completely,”), use "Word-Usage/Clarity."

      # Ambiguous case:

      Before: "Self-driving cars are the future."
      After: "Self-driving vehicles represent the future."

      → This is **Word-Usage/Clarity**, because it swaps terms for clarity without adding detail.

- **Claim/Ideas vs. Warrant/Reasoning/Backing**: A main claim is "Claims/Ideas," while supporting arguments to an already written claim are "Warrant/Reasoning/Backing."  
- **General Content vs. Warrant/Reasoning/Backing**: If it expresses the author's stance, classify as "Warrant/Reasoning/Backing"; otherwise, as "General Content."  
- **Evidence vs. Warrant/Reasoning/Backing**: Factual support (e.g., citations, examples) is "Evidence"; otherwise, it's "Warrant/Reasoning/Backing."   
- **Organization vs. General Content Development**: Reordering sentences is "Organization"; significant content changes are "General Content."  

You **must** consider previous messages in the thread as part of your context. Many revisions cannot be accurately classified in isolation especially to see the links between the claims, evidences and reasoning categories.

**Output Format:**
Your response **must** follow this exact JSON format:  
```json
{
      "reasoning": "<Step-by-step explanation of the classification>",
      "classification": "category chosen"
}""",
  tools=[],
  model="gpt-4o",
)

In [ ]:
assistant_first = client.beta.assistants.create(
  name="ArgRewrite-try-2-step2-surface",
  instructions="""You are evaluating text revisions by comparing different states of a text (before and after a revision). The considered revision has been classified as being a surface revision. You should now subclassify it.

**Contextual Awareness:**  
You are evaluating a *sequence* of revisions within the same essay. It is essential to consider earlier parts of the essay to accurately determine the purpose of a revision. Always reason based on the surrounding context and how the revision fits into the overall argumentative structure.


**Prompt students answered:**  
Write an op-ed for the *Pittsburgh City Paper* on whether self-driving cars benefit society. Briefly outline their advantages and disadvantages, then take a stance—either for or against—supporting it with strong evidence, explanations, and a counter-argument. Ensure clarity, precision, and proper organization.


For each revision of the text do this classification task : 
**Classification Task:**  
Provide a step-by-step reasoning (Chain of Thought) before concluding with one of the following categories for each text given as input:

- **Word-Usage/Clarity**: Edits that improve fluency, clarity, or word choice, without changing factual content or specificity. Use "Word-Usage/Clarity" for fluency improvements, synonyms, or idiomatic rewrites that preserve meaning.
- Example:  
      - Before: "Technology is the nourishment of civilizations."  
      - After: "Technology is the backbone of civilizations." 

- **Conventions/Grammar/Spelling**: Corrections of spelling and grammatical errors or convent.
- Example:  
      - Before: "Big corporations stick together and attack on one another."  
      - After: "Big corporations stick together and attack one another."

- **Organization**: Structural modifications, such as rearranging sentences by splitting them or reunite them.
- Example:  
      - Before: "In my opinion, I support self-driving cars. The technology will have positive impacts on the society."  
      - After: "In my opinion, I support self-driving cars because the technology will have positive impacts on the society."


**Guidelines for Ambiguous Cases:**  
If you hesitate between multiple revision purposes, follow these rules:  

- **Conventions/Grammar/Spelling vs. Word-Usage/Clarity**: If only grammar/spelling is corrected, use "Conventions/Grammar/Spelling"; otherwise, use "Word-Usage/Clarity."  

You **must** consider previous messages in the thread as part of your context.

**Output Format:**
Your response **must** follow this exact JSON format:  
```json
{
       "reasoning": "<Step-by-step explanation of the classification>",
      "classification": "category chosen"
}""",
  tools=[],
  model="gpt-4o",
)

# Essai 3 : Reverse

In [ ]:
assistant_first = client.beta.assistants.create(
  name="ArgRewrite-try-3",
  instructions="""You are evaluating text revisions by comparing different states of a text (before and after a revision).

**Contextual Awareness:**  
You are evaluating a *sequence* of revisions within the same essay. It is essential to consider earlier parts of the essay to accurately determine the purpose of a revision. Always reason based on the surrounding context and how the revision fits into the overall argumentative structure.


**Prompt students answered:**  
Write an op-ed for the *Pittsburgh City Paper* on whether self-driving cars benefit society. Briefly outline their advantages and disadvantages, then take a stance—either for or against—supporting it with strong evidence, explanations, and a counter-argument. Ensure clarity, precision, and proper organization.


For each revision of the text do this classification task : 
**Classification Task:**  
Provide a step-by-step reasoning (Chain of Thought) before concluding with one of the following categories for each text given as input:

- **Word-Usage/Clarity**: Edits that improve fluency, clarity, or word choice, without changing factual content or specificity. Use "Word-Usage/Clarity" for fluency improvements, synonyms, or idiomatic rewrites that preserve meaning.
- Example:  
      - Before: "Technology is the nourishment of civilizations."  
      - After: "Technology is the backbone of civilizations." 

- **Conventions/Grammar/Spelling**: Corrections of spelling and grammatical errors or convent.
- Example:  
      - Before: "Big corporations stick together and attack on one another."  
      - After: "Big corporations stick together and attack one another."

- **General Content Development**: Significant additions, deletions, or rewording of content without a clear argumentative function.
- Example:  
      - Before: "Self-driving cars will one day be the same."  
      - After: "Self-driving cars will one day be the same - once thought impossible, but eventually taken for granted."


- **Precision**: Edits that add or reduce quantifiable, factual, or definitional detail, affecting how specific a statement is and altering its meaning. Use "Precision" only when an edit introduces or removes specific factual detail (e.g., names, quantities, types). If a change slightly broadens or narrows specificity without introducing or removing factual detail, classify it as "Word-Usage/Clarity."
- Example:  
      - Before: "Self-driving cars are helpful."
      - After: "Self-driving cars reduce accidents by 90%."
Use "Word-Usage/Clarity" when edits add stylistic or rhetorical specificity, such as clarifying tone or phrasing, unless the edit introduces concrete facts, types, quantities, or definitions.
- Example:  
      - Before: "make a decision"  
      - After: "make a human judgment decision"

      → This is **Word-Usage/Clarity**. While the revision adds a descriptive qualifier, it does not introduce factual or technical specificity.


- **Organization**: Structural modifications, such as rearranging sentences by splitting them or reunite them.
- Example:  
      - Before: "In my opinion, I support self-driving cars. The technology will have positive impacts on the society."  
      - After: "In my opinion, I support self-driving cars because the technology will have positive impacts on the society."

- **Warrant/Reasoning/Backing**: Changes in explanations or logical support for a claim, related to the prompt students answered to.
- Example:  
      - Before: "By reducing traffic, gas expenses would be lower."  
      - After: "Self-driving cars will have the ability to coordinate with each other to prevent and reduce traffic, which means gas expenses would be lower."

- **Evidence**: Additions or modifications of factual information to support an argument that is link to the prompt students answered to.
- Example:  
      - Before: "A lot of expensive machinery will be needed to make a self-driving car."  
      - After: "A lot of expensive machinery, like sensors and cameras, will be needed to make a self-driving car."


- **Claims/Ideas**: Edits that modify (add or delete) a major claim or idea of the essay related to the prompt students answered to. Make sure the revision is not related to a previous claim before choosing this type, if it is choose Warrant/Reasoning/Backing instead.
- Example:  
      - Before: "Overall, self-driving cars are infiltrating our society at a rather rapid rate."  
      - After: "Overall, self-driving cars are immigrating into our society with many benefits."

- **Rebuttal/Reservation**: Edits related to refuting an opposing argument.
- Example:  
      - Before: "Self driving cars are the subject of much scrutiny."  
      - After: "Self driving cars are not perfect yet, and there are still many issues to be worked out."


**Guidelines for Ambiguous Cases:**  
If you hesitate between multiple revision purposes, follow these rules:  

- **Precision vs. Word-Usage/Clarity**: Use "Precision" only if the revision introduces or removes **factual**, **technical**, or **quantifiable** detail. If the change improves clarity or adds rhetorical nuance (e.g., “completely,” “human judgment”), use "Word-Usage/Clarity."

      # Ambiguous case:

      Before: "Self-driving cars are the future."
      After: "Self-driving vehicles represent the future."

      → This is **Word-Usage/Clarity**, because it swaps terms for clarity without adding detail.

- **Claim/Ideas vs. Warrant/Reasoning/Backing**: A main claim is "Claims/Ideas," while supporting arguments to an already written claim are "Warrant/Reasoning/Backing."  
- **General Content vs. Warrant/Reasoning/Backing**: If it expresses the author's stance, classify as "Warrant/Reasoning/Backing"; otherwise, as "General Content."  
- **Evidence vs. Warrant/Reasoning/Backing**: Factual support (e.g., citations, examples) is "Evidence"; otherwise, it's "Warrant/Reasoning/Backing."  
- **Conventions/Grammar/Spelling vs. Word-Usage/Clarity**: If only grammar/spelling is corrected, use "Conventions/Grammar/Spelling"; otherwise, use "Word-Usage/Clarity."  
- **Claim/Idea vs. Word-Usage/Clarity**: If a change impacts the essay’s main argument, classify as "Claim/Ideas."  
- **Organization vs. General Content Development**: Reordering sentences is "Organization"; significant content changes are "General Content."  

You **must** consider previous messages in the thread as part of your context. Many revisions cannot be accurately classified in isolation especially to see the links between the claims, evidences and reasoning categories.

**Output Format:**
Your response **must** follow this exact JSON format:  
```json
{
      "classification": "category chosen",
      "reasoning": "<Step-by-step explanation of the classification>"
      
}""",
  tools=[],
  model="gpt-4o",
)

# Predict and evaluate

In [ ]:
# List all assistants
assistants = client.beta.assistants.list()

# Print their IDs
for assistant in assistants.data:
    print(f"Name: {assistant.name}, ID: {assistant.id}")

In [ ]:
assistant_id = "your_assistant_id"
# or in case of multiple steps
assistant_id_step1 = "your_assistant_id"
assistant_id_step2_content = "your_assistant_id"
assistant_id_step2_surface = "your_assistant_id"

In [ ]:

def classify_with_assistants_full_text(text_before, text_after, thread, previously_classified):
    while True:
        runs = client.beta.threads.runs.list(thread_id=thread.id)
        active_runs = [r for r in runs.data if r.status in ["queued", "in_progress"]]
        
        if not active_runs:
            break  # No active runs, proceed
        print("Waiting for active run to complete...")
        time.sleep(2)  # Wait and check again

    # Step 1: Request Meaning Classification Assistant
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=f"Here is previously classified text belonging to the same student's answer : {previously_classified}. Classify this revision:\nBefore: {text_before}\nAfter: {text_after}\nReturn JSON output."
    )

    # Run first assistant
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

    # Wait for completion
    while run.status in ["queued", "in_progress"]:
        time.sleep(2)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve response
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(messages)
    response_text = messages.data[0].content[0].text.value
    print(response_text)

    try:
        clean_response_text = response_text.replace("```json", "").replace("```", "").strip()
        response = json.loads(clean_response_text)
        category = response['classification']
        reasoning_1 = response['reasoning']
        
    except json.JSONDecodeError:
        print("Error parsing first response:", response_text)
        exit()


    print("Step 1 Classification:", category)

    
    return {"classification": category, "reasoning": reasoning_1}

In [ ]:

def classify_with_assistants_multiple_steps(text_before, text_after, thread_main, thread_content, thread_surface, previously_classified):
    while True:
        runs = client.beta.threads.runs.list(thread_id=thread_main.id)
        active_runs = [r for r in runs.data if r.status in ["queued", "in_progress"]]
        
        if not active_runs:
            break  # No active runs, proceed
        print("Waiting for active run to complete...")
        time.sleep(2)  # Wait and check again

    # Step 1: Request Meaning Classification Assistant
    message = client.beta.threads.messages.create(
        thread_id=thread_main.id,
        role="user",
        content=f"Classify this revision:\nBefore: {text_before}\nAfter: {text_after}\nReturn JSON output."
    )

    # Run first assistant
    run = client.beta.threads.runs.create(thread_id=thread_main.id, assistant_id=assistant_id_step1)

    # Wait for completion
    while run.status in ["queued", "in_progress"]:
        time.sleep(2)
        run = client.beta.threads.runs.retrieve(thread_id=thread_main.id, run_id=run.id)

    # Retrieve response
    messages = client.beta.threads.messages.list(thread_id=thread_main.id)
    print(messages)
    response_text = messages.data[0].content[0].text.value
    print(response_text)

    try:
        clean_response_text = response_text.replace("```json", "").replace("```", "").strip()
        response = json.loads(clean_response_text)
        main_category = response['classification']
        reasoning_1 = response['reasoning']
        
    except json.JSONDecodeError:
        print("Error parsing first response:", response_text)
        exit()


    print("Step 1 Classification:", main_category)

    if main_category=="Surface changes" or main_category == "surface changes":
        while True:
            runs = client.beta.threads.runs.list(thread_id=thread_surface.id)
            active_runs = [r for r in runs.data if r.status in ["queued", "in_progress"]]
            
            if not active_runs:
                break  # No active runs, proceed
            print("Waiting for active run to complete...")
            time.sleep(2)  # Wait and check again

        # Step 1: Request Meaning Classification Assistant
        message = client.beta.threads.messages.create(
            thread_id=thread_surface.id,
            role="user",
            content=f"Here is previously classified text belonging to the same student's answer : {previously_classified}. Classify this revision:\nBefore: {text_before}\nAfter: {text_after}\nReturn JSON output."
        )

        # Run first assistant
        run = client.beta.threads.runs.create(thread_id=thread_surface.id, assistant_id=assistant_id_step2_surface)

        # Wait for completion
        while run.status in ["queued", "in_progress"]:
            time.sleep(2)
            run = client.beta.threads.runs.retrieve(thread_id=thread_surface.id, run_id=run.id)

        # Retrieve response
        messages = client.beta.threads.messages.list(thread_id=thread_surface.id)
        print(messages)
        response_text = messages.data[0].content[0].text.value
        print(response_text)

        try:
            clean_response_text = response_text.replace("```json", "").replace("```", "").strip()
            response = json.loads(clean_response_text)
            category = response['classification']
            reasoning_2 = response['reasoning']
            
        except json.JSONDecodeError:
            print("Error parsing first response:", response_text)
            exit()


    elif main_category=="Content changes" or main_category=="content changes":
        while True:
            runs = client.beta.threads.runs.list(thread_id=thread_content.id)
            active_runs = [r for r in runs.data if r.status in ["queued", "in_progress"]]
            
            if not active_runs:
                break  # No active runs, proceed
            print("Waiting for active run to complete...")
            time.sleep(2)  # Wait and check again

        # Step 1: Request Meaning Classification Assistant
        message = client.beta.threads.messages.create(
            thread_id=thread_content.id,
            role="user",
            content=f"Here is previously classified text belonging to the same student's answer : {previously_classified}. Classify this revision:\nBefore: {text_before}\nAfter: {text_after}\nReturn JSON output."
        )

        # Run first assistant
        run = client.beta.threads.runs.create(thread_id=thread_content.id, assistant_id=assistant_id_step2_content)

        # Wait for completion
        while run.status in ["queued", "in_progress"]:
            time.sleep(2)
            run = client.beta.threads.runs.retrieve(thread_id=thread_content.id, run_id=run.id)

        # Retrieve response
        messages = client.beta.threads.messages.list(thread_id=thread_content.id)
        print(messages)
        response_text = messages.data[0].content[0].text.value
        print(response_text)

        try:
            clean_response_text = response_text.replace("```json", "").replace("```", "").strip()
            response = json.loads(clean_response_text)
            category = response['classification']
            reasoning_2 = response['reasoning']
            
        except json.JSONDecodeError:
            print("Error parsing first response:", response_text)
            exit()

    else:
        print("ERROR")
        exit()
    
    return {"classification": category, "reasoning_step1": reasoning_1, "reasoning_step2": reasoning_2}

In [ ]:
def predict(dataf, save_path="path_to_save_predictions.csv"):
    # Load existing predictions if the file exists
    if os.path.exists(save_path):
        existing_df = pd.read_csv(save_path, index_col=0)
        completed_indices = set(existing_df.index)  # Indices of already processed rows
        # Filter the original dataframe to only include rows that have not been processed yet
        dataf = dataf[~dataf.index.isin(completed_indices)]
    else:
        completed_indices = set()

    # Process the remaining rows
    prev_text_id = -1
    previously_classified = ""
    for index, row in dataf.iterrows():
        try:
            if row["Doc Id"]!= prev_text_id:
                thread_main = client.beta.threads.create()
                thread_content = client.beta.threads.create()
                thread_surface = client.beta.threads.create()
                prev_text_id = row["Doc Id"]
                previously_classified = ""

            #adapt to not multiple steps model if needed
            prediction = classify_with_assistants_multiple_steps(row['Text Before'], row['Text After'], thread_main, thread_content, thread_surface, previously_classified)
            previously_classified+= f"""Previous text : {row['Text After']}, Label given : {prediction['classification']};\n"""
            dataf.loc[index, "prediction"] = prediction["classification"]
            if "reasoning_step1" in prediction:
                dataf.loc[index, "reasoning_step1"] = prediction["reasoning_step1"]
            if "reasoning_step2" in prediction:
                dataf.loc[index, "reasoning_step2"] = prediction["reasoning_step2"]
            dataf.loc[[index]].to_csv(save_path, mode="a", header=not os.path.exists(save_path), index=True)

            # Mark this index as completed
            completed_indices.add(index)

        except Exception as e:
            print(f"Error at index {index}: {e}")
            break  # Stop execution if needed

    return dataf

In [ ]:
predict(test_df)

In [4]:
def simplify_labels(y):
    y_simplified = []
    surface_labels = [
    "word-usage/clarity",
    "conventions/grammar/spelling",
    "organization"        
    ]
    content_labels = [
    "general content development",
    "precision",
    "warrant/reasoning/backing",
    "evidence",
    "claims/ideas",
    "rebuttal/reservation"
    ]
    for label in y:
        if label in surface_labels:
            y_simplified.append("surface")
        elif label in content_labels:
            y_simplified.append("content")
        else:
            print("issue")
            print(label)
            y_simplified.append("none")
    return y_simplified

In [1]:
def mapping_categories(df):
    # Define the mapping
    mapping = {
        "Word-Usage/Clarity": "Clarity",
        "Conventions/Grammar/Spelling": "Grammar",
        "Evidence": "Fact/Evidence",
        "Claims/Ideas": "Claim",
        "Claim/Ideas": "Claim",
        "Warrant/Reasoning/Backing": "Claim",
        "Rebuttal/Reservation": "Claim",
        "Organization": "Clarity",
        "Precision": "Other",
        "General Content Development": "Other"
    }

    # Check for unmapped values in both columns
    for col in ["Label", "prediction"]:
        unmatched = set(df[col].unique()) - set(mapping.keys())
        if unmatched:
            raise ValueError(f"Unmatched values in column '{col}': {unmatched}")

    # Apply the mapping
    df["Label"] = df["Label"].map(mapping)
    df["prediction"] = df["prediction"].map(mapping)
    return df

In [ ]:
def evaluate(results):
    results["prediction"] = results["prediction"].str.lower()
    results["Label"] = results["Label"].str.lower()
    
    y_true = results["Label"].to_numpy()
    y_pred = results["prediction"].to_numpy()


    class_labels = [
    "word-usage/clarity",
    "conventions/grammar/spelling",
    "general content development",
    "precision",
    "organization",
    "warrant/reasoning/backing",
    "evidence",
    "claims/ideas",
    "rebuttal/reservation"
    ]

    # Per-label metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=class_labels)
    accuracy = accuracy_score(y_true, y_pred)

    # Macro (unweighted) scores
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', labels=class_labels)

    # Macro (weighted) scores
    precision_w, recall_w, f1_w, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', labels=class_labels)

    # Error pairs
    error_pairs = Counter((t, p) for t, p in zip(y_true, y_pred) if t != p)
    sorted_errors = error_pairs.most_common()
    for pair, count in sorted_errors:
        print(f"{pair}: {count}")

        # Convert to a DataFrame for readability
    metrics_df = pd.DataFrame({"Label": class_labels, "Accuracy" : accuracy, "Precision": precision, "Recall": recall, "F1-score": f1})
    # Display metrics

    print(metrics_df)
    print(f"Macro Unweighted Metrics:\nPrecision :{precision_macro} \nRecall : {recall_macro} \nF1-Score : {f1_macro}")
    print(f"Weighted Metrics:\nPrecision :{precision_w} \nRecall : {recall_w} \nF1-Score : {f1_w}")


    y_true_simplified = simplify_labels(y_true)
    y_pred_simplied = simplify_labels(y_pred)

    simplify_class_labels = [
        "surface",
        "content",
        "none"
    ]
    # Compute per-label precision, recall, and F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(y_true_simplified, y_pred_simplied, average=None, labels=simplify_class_labels)
    # Convert to a DataFrame for readability
    metrics_simplified_df = pd.DataFrame({"Label": simplify_class_labels, "Precision": precision, "Recall": recall, "F1-score": f1})
    print(metrics_simplified_df)

    kappa = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    print(f"Cohen's Kappa for fine granularity : {kappa:.4f}")

   
    kappa = cohen_kappa_score(y_true_simplified, y_pred_simplied)
    print(f"Cohen's Kappa for coarse granularity : {kappa:.4f}")
    







In [ ]:
def evaluate_ruan(results):
    results = mapping_categories(results)
    results["prediction"] = results["prediction"].str.lower()
    results["Label"] = results["Label"].str.lower()
    
    y_true = results["Label"].to_numpy()
    y_pred = results["prediction"].to_numpy()
    
    class_labels = [
    "clarity",
    "grammar",
    "claim",
    "other",
    "fact/evidence",
    ]

    # Per-label metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None, labels=class_labels)
    accuracy = accuracy_score(y_true, y_pred)

    # Macro (unweighted) scores
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', labels=class_labels)

    # Macro (weighted) scores
    precision_w, recall_w, f1_w, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', labels=class_labels)
    # Error pairs
    error_pairs = Counter((t, p) for t, p in zip(y_true, y_pred) if t != p)
    sorted_errors = error_pairs.most_common()
    for pair, count in sorted_errors:
        print(f"{pair}: {count}")

    # Convert to a DataFrame for readability
    metrics_df = pd.DataFrame({"Label": class_labels, "Accuracy" : accuracy, "Precision": precision, "Recall": recall, "F1-score": f1})
    # Display metrics
    print(metrics_df)
    print(f"Macro Unweighted Metrics:\nPrecision :{precision_macro} \nRecall : {recall_macro} \nF1-Score : {f1_macro}")
    print(f"Weighted Metrics:\nPrecision :{precision_w} \nRecall : {recall_w} \nF1-Score : {f1_w}")


    kappa = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    print(f"Cohen's kappa: {kappa:.4f}")

    


/home/nbl/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# DSPy

In [ ]:
import dspy
import tiktoken
import pandas as pd


from collections import Counter
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from typing import Literal

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score


In [20]:
complete_train_df = pd.read_csv("/home/nbl/llm-notebooks/train_set_argRewrite_complete.csv")

gss = GroupShuffleSplit(n_splits=1, test_size=0.06, random_state=42)
train_idxs, test_idxs = next(gss.split(complete_train_df, groups=complete_train_df["Doc Id"]))

train_df_dspy = complete_train_df.iloc[test_idxs]

print("Training set size:", len(train_df_dspy))

Training set size: 158


In [78]:
train_df_dspy.to_csv("training_set_dspy.csv")

In [2]:
train_df_dspy = pd.read_csv("/home/nbl/llm-notebooks/training_set_dspy.csv")

In [ ]:
lm = dspy.LM('openai/o3-mini', api_key='your_api_key', temperature=1.0, max_tokens=5000)
dspy.configure(lm=lm)


# Your 9 labels
labels = [
    "word-usage/clarity",
    "conventions/grammar/spelling",
    "organization",
    "general content development",
    "precision",
    "warrant/reasoning/backing",
    "evidence",
    "claims/ideas",
    "rebuttal/reservation"
]

# Define a DSPy module for classification
class RevisionClassifier(dspy.Signature):
    """Classify the revision into one of 9 categories based on before/after text."""
    text_before_revision: str = dspy.InputField()
    text_after_revision: str = dspy.InputField()
    label: Literal['word-usage/clarity', 'conventions/grammar/spelling', 'organization', 'precision', 'general content development', 'rebuttal/reservation', 'claims/ideas', 'evidence', 'warrant/reasoning/backing'] = dspy.OutputField()

# Wrap it into a DSPy program
class ClassifyRevision(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classifier = dspy.ChainOfThought(RevisionClassifier)

    def forward(self, text_before_revision, text_after_revision):
        return self.classifier(text_before_revision=text_before_revision, text_after_revision=text_after_revision)

# Load your dataset as DSPy Examples
def load_examples(df):
    return [
        dspy.Example(
            text_before_revision=  row["Text Before"],
            text_after_revision= row["Text After"],
            label= row["Label"].lower()
        ).with_inputs("text_before_revision", "text_after_revision")
        for _, row in df.iterrows()
    ]

def validate_answer(example, pred, trace=None):
    return example.label.lower() == pred.label.lower()

# Optional: optimize prompt with few-shot examples
def optimize_with_bootstrap(train_examples):
    teleprompter = dspy.BootstrapFewShotWithRandomSearch(metric=validate_answer, num_candidate_programs=10)

    program = ClassifyRevision()
    optimized_program = teleprompter.compile(program, trainset=train_examples)

    return optimized_program


In [ ]:


# Assuming you have a DataFrame called df with 'text_before', 'text_after', 'label'
df = train_df_dspy
test_df = pd.read_csv("/home/nbl/llm-notebooks/test_set_argRewrite_complete.csv")
train_data = load_examples(df)
test_data = load_examples(test_df)


# Train (optimize prompt + weights)
optimized_model = optimize_with_bootstrap(train_data)




In [ ]:
from sklearn.metrics import classification_report
import pandas as pd

# Collect predictions
preds = []
golds = []

for ex in test_data:
    output = optimized_model.forward(**ex.inputs)
    pred = output.label
    gold = ex.outputs["label"]

    preds.append(pred)
    golds.append(gold)

    print(f"Gold: {gold} | Pred: {pred}")

# Save predictions to CSV
df_preds = pd.DataFrame({
    "Label": golds,
    "prediction": preds
})
df_preds.to_csv("revision_predictions.csv", index=False)



In [ ]:
evaluate(df_preds)